In [2]:
from ultralytics import YOLO
import cv2
import math
import json
import os

#to set the winwod size
DESIRED_WIDTH = 1280
DESIRED_HEIGHT = 720

#LOad  the extracted coordinates
try:
    with open("hoinu1.json", "r") as f:
        right_frames = json.load(f)
    with open("hoinu2.json", "r") as f:
        left_frames = json.load(f)
except FileNotFoundError:
    print("Error: 'hoinu1.json' or 'hoinu2.json' not found.")
    exit()
except json.JSONDecodeError:
    print("Error: Could not decode JSON.")
    exit()

#check if we have enough frames
if len(right_frames) <= 33 or len(left_frames) <= 1:
    print("Error: Not enough frames in datasets.")
    exit()

right_landmarks = right_frames[33]["landmarks"]
left_landmarks = left_frames[0]["landmarks"]

#Ensure enough landmarks are present in the chosen reference frames
if len(right_landmarks) < 13 or len(left_landmarks) < 13:
    print("Error: Not enough landmarks in reference frames.")
    exit()
    
"""Important indices for comparison using YOLOv8's COCO 17-point keypoint set
 COCO 17 Keypoint Mapping:
0: Nose
5: Left Shoulder
6: Right Shoulder
9: Left Wrist
10: Right Wrist"""
important_indices = [0, 10, 9]

def calc_distance(p1, p2):
    #Calculates Euclidean distance between two 2D points.
    return math.sqrt((p1["x"] - p2["x"]) ** 2 + (p1["y"] - p2["y"]) ** 2)

def normalize_landmarks(landmarks):
    """ Normalizes landmark coordinates based on shoulder positions to make pose comparison
    robust to scale and translation.
    
    [5] is for left shoilder and [6] is for right one
    """
    if len(landmarks) < 7:
        return None
    try:
        # Ensure the crucial shoulder landmarks exist and have 'x', 'y' keys.
        # Check if indices 5 and 6 are within the bounds of the landmarks list
        if 5 not in range(len(landmarks)) or 6 not in range(len(landmarks)):
            return None
        left_shoulder = landmarks[5]
        right_shoulder = landmarks[6]
        if not all(k in left_shoulder for k in ['x', 'y']) or not all(k in right_shoulder for k in ['x', 'y']):
            return None
        center_x = (left_shoulder["x"] + right_shoulder["x"]) / 2
        center_y = (left_shoulder["y"] + right_shoulder["y"]) / 2
        scale = math.sqrt((left_shoulder["x"] - right_shoulder["x"]) ** 2 + (left_shoulder["y"] - right_shoulder["y"]) ** 2)
        if scale == 0:
            scale = 1e-6
        normalized = []
        for lm in landmarks:
            if 'x' in lm and 'y' in lm:
                norm_x = (lm["x"] - center_x) / scale
                norm_y = (lm["y"] - center_y) / scale
                normalized.append({"x": norm_x, "y": norm_y})
            else:
                return None
        return normalized
    except Exception:
        return None

norm_right = normalize_landmarks(right_landmarks)
norm_left = normalize_landmarks(left_landmarks)

if norm_right is None or norm_left is None:
    print("Error: Reference pose normalization failed.")
    exit()
    
#load yolov8 model
try:
    model = YOLO("yolov8s-pose.pt")
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

#open cam
cap = cv2.VideoCapture(1)#set it as 0 for laptop webcam
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

#adjust cam resolution
cap.set(cv2.CAP_PROP_FRAME_WIDTH, DESIRED_WIDTH)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, DESIRED_HEIGHT)

print("Starting webcam feed. Press 'q' to quit.")

window_name = "YOLOv8s Pose Cheating Detection"
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_name, DESIRED_WIDTH, DESIRED_HEIGHT)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame. Exiting...")
        break

    #perfform the actual objectuve(pose estimation)
    results = model(frame, verbose=False)[0]
    cheating = False
    turn_direction = ""
    person_for_normalization_found = False

    if results.keypoints is not None and len(results.keypoints) > 0:
        #find the perosn with the most keypoints
        best_person_keypoints_data = None
        max_keypoints = 0

        for person_kp_obj in results.keypoints:
            current_person_keypoints_data = person_kp_obj.data.cpu().numpy()
            if current_person_keypoints_data.ndim == 3 and current_person_keypoints_data.shape[0] == 1:
                current_person_keypoints_data = current_person_keypoints_data[0]
            if len(current_person_keypoints_data) > max_keypoints:
                max_keypoints = len(current_person_keypoints_data)
                best_person_keypoints_data = current_person_keypoints_data

        if best_person_keypoints_data is not None:
            person_for_normalization_found = True
            height, width = frame.shape[:2]
            live_pts = []
            for kp in best_person_keypoints_data:
                x_norm = kp[0] / width
                y_norm = kp[1] / height
                live_pts.append({"x": x_norm, "y": y_norm})

            if len(live_pts) >= 7:
                norm_live = normalize_landmarks(live_pts)
                if norm_live is None:
                    turn_direction = "Normalization failed."
                else:
                    right_score = 0
                    left_score = 0
                    for i in important_indices:
                        if i < len(norm_right) and i < len(norm_left) and i < len(norm_live):
                            ref_r = norm_right[i]
                            ref_l = norm_left[i]
                            live_pt = norm_live[i]
                            right_score += calc_distance(live_pt, ref_r)
                            left_score += calc_distance(live_pt, ref_l)
                        else:
                            print(f"Warning: Important index {i} out of bounds for some landmark sets. Skipping comparison for this index.")
                            continue
                            
                    if right_score < 0.75:#accuracy treshold
                        cheating = True
                        turn_direction = f"Right Turn Detected (Cheating) - Score: {right_score:.2f}"
                    elif left_score < 0.75: #accuracy treshold
                        cheating = True
                        turn_direction = f"Left Turn Detected (Cheating) - Score: {left_score:.2f}"
                    else:
                        turn_direction = f"Normal - Right Score: {right_score:.2f}, Left Score: {left_score:.2f}"
            else:#if th person dosnet have enough key points
                turn_direction = "Not enough keypoints."
        else:#no viavle perosn
            turn_direction = "No viable person data."
    else:#no one detected
        turn_direction = "No person detected."

    # Display cheating status on the frame
    text_color = (0, 0, 255) if cheating else (0, 255, 0)
    cv2.putText(frame, turn_direction, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 3)

    annotated_frame = results.plot()
    cv2.imshow(window_name, annotated_frame)

    #inesrt any key here to exist,adjust on ur own accord
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
#byebye
cap.release()
cv2.destroyAllWindows()


Starting webcam feed. Press 'q' to quit.
